In Databricks with Delta Lake, deletion vectors are used for handling deletes in a more efficient and manageable manner, especially in big data scenarios. When you delete data in Delta Lake, instead of physically removing rows, Delta Lake creates a deletion vector. This vector contains information about which files and which parts of those files have been logically deleted. Here's why this approach is beneficial:

###Atomic Operations: 
Deletion vectors ensure that deletes are atomic. This means they either fully succeed or fully fail, avoiding partial deletions that could lead to inconsistent data.

###Efficient Storage: 
Instead of rewriting entire files or partitions, Delta Lake just marks the rows as deleted in the deletion vector. This reduces overhead and improves performance by minimizing the amount of data that needs to be rewritten.

###Transaction History: 
Delta Lake maintains a transaction log that records all operations (including deletes). This log allows for easy auditability and rollbacks if necessary.

###Time Travel: 
Deletion vectors are also integral to Delta Lake's time travel feature, which enables you to query data as it appeared at any point in time, including before deletions.

To handle deletion vectors in PySpark with Delta Lake, you typically use Delta Lake's native APIs. For instance, you can perform delete operations using DeltaTable API in PySpark, which ensures that deletion vectors are properly managed and transactions are atomic. This ensures data consistency and integrity while maintaining efficiency in data operations.

In [0]:
from pyspark.sql import SparkSession
from delta.tables import DeltaTable

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Delta Lake Deletion Vectors Example") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Example DataFrame
data = [(1, 'John'), (2, 'Jane'), (3, 'Doe')]
df = spark.createDataFrame(data, ["id", "name"])

# Write DataFrame to Delta Lake
df.write.format("delta").mode("overwrite").save("/delta-table")

# Read Delta table
delta_table = DeltaTable.forPath(spark, "/delta-table")

# Perform delete operation
condition = "id = 2"
delta_table.delete(condition)

# Read the Delta table after deletion
delta_df = spark.read.format("delta").load("/delta-table")
delta_df.display()

# Delta table time travel example - accessing data before deletion
old_data = delta_table.history().filter("version < 2").select("id", "name")
old_data.display()


id,name
1,John
3,Doe


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3516885369338641>:30
     27 delta_df.display()
     29 # Delta table time travel example - accessing data before deletion
---> 30 old_data = delta_table.history().filter("version < 2").select("id", "name")
     31 old_data.display()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:3023, in DataFrame.select(self, *cols)
   2978 def select(self, *cols: "ColumnOrName") -> "DataFrame":  # type: ignore[misc]
   2979     """Projects a set of expressions a

###In this example:

We create a Spark session and define a sample DataFrame df with some data (id, name).

The DataFrame is written to Delta Lake (/delta-table).

We access the Delta table using DeltaTable.forPath() and perform a delete operation based on a condition (id = 2).

After deletion, we read the Delta table again (delta_df) and display its contents.

The history() function of DeltaTable allows us to view the transaction history. We filter the history to show data before the deletion (version < 2).

This example illustrates how Delta Lake manages deletion vectors internally and how you can work with them using PySpark APIs (DeltaTable).